## Starter Code to Explore The Dataset 

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [21]:
# Convert the csv file into a Pandas DataFrame
primary_df =pd.read_csv("Resources/Worldwide-Earthquake-database.csv", index_col=0)

# Display sample data
primary_df.head()

,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,EQ_MAG_MW,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
I_D,,,,,,,,,,,,,,,,,,,,,
1,No,-2150,NaN,NaN,NaN,NaN,,NaN,7.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yes,-2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,-2000,NaN,NaN,NaN,NaN,NaN,18.0,7.1,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
5877,Yes,-1610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
8,No,-1566,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#Total number of row and columns 
primary_df.shape

(6193, 46)

In [18]:
#Printing the columns' name
primary_df.columns

Index(['FLAG_TSUNAMI', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND',
       'FOCAL_DEPTH', 'EQ_PRIMARY', 'EQ_MAG_MW', 'EQ_MAG_MS', 'EQ_MAG_MB',
       'EQ_MAG_ML', 'EQ_MAG_MFA', 'EQ_MAG_UNK', 'INTENSITY', 'COUNTRY',
       'STATE', 'LOCATION_NAME', 'LATITUDE', 'LONGITUDE', 'REGION_CODE',
       'DEATHS', 'DEATHS_DESCRIPTION', 'MISSING', 'MISSING_DESCRIPTION',
       'INJURIES', 'INJURIES_DESCRIPTION', 'DAMAGE_MILLIONS_DOLLARS',
       'DAMAGE_DESCRIPTION', 'HOUSES_DESTROYED',
       'HOUSES_DESTROYED_DESCRIPTION', 'HOUSES_DAMAGED',
       'HOUSES_DAMAGED_DESCRIPTION', 'TOTAL_DEATHS',
       'TOTAL_DEATHS_DESCRIPTION', 'TOTAL_MISSING',
       'TOTAL_MISSING_DESCRIPTION', 'TOTAL_INJURIES',
       'TOTAL_INJURIES_DESCRIPTION', 'TOTAL_DAMAGE_MILLIONS_DOLLARS',
       'TOTAL_DAMAGE_DESCRIPTION', 'TOTAL_HOUSES_DESTROYED',
       'TOTAL_HOUSES_DESTROYED_DESCRIPTION', 'TOTAL_HOUSES_DAMAGED',
       'TOTAL_HOUSES_DAMAGED_DESCRIPTION'],
      dtype='object')

In [25]:
#Narrow down the columns
worldwide_earthquake_df = primary_df[['FLAG_TSUNAMI', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'FOCAL_DEPTH', 
                                      'INTENSITY', 'COUNTRY', 'LATITUDE', 'LONGITUDE', 'TOTAL_DEATHS',
                                      'TOTAL_INJURIES']] 

In [26]:
# Display sample of narrowed down data
worldwide_earthquake_df.head(10)

,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,FOCAL_DEPTH,INTENSITY,COUNTRY,LATITUDE,LONGITUDE,TOTAL_DEATHS,TOTAL_INJURIES
I_D,,,,,,,,,,,,
1,No,-2150,NaN,NaN,NaN,NaN,NaN,JORDAN,31.1,35.5,NaN,NaN
2,Yes,-2000,NaN,NaN,NaN,NaN,10.0,SYRIA,35.683,35.8,NaN,NaN
3,No,-2000,NaN,NaN,NaN,18.0,10.0,TURKMENISTAN,38,58.2,1.0,NaN
5877,Yes,-1610,NaN,NaN,NaN,NaN,NaN,GREECE,36.4,25.4,NaN,NaN
8,No,-1566,NaN,NaN,NaN,NaN,10.0,ISRAEL,31.5,35.3,NaN,NaN
11,No,-1450,NaN,NaN,NaN,NaN,10.0,ITALY,35.5,25.5,NaN,NaN
9712,Yes,-1365,NaN,NaN,NaN,NaN,NaN,SYRIA,35.683,35.8,NaN,NaN
12,No,-1250,NaN,NaN,NaN,NaN,NaN,ISRAEL,32,35.5,NaN,NaN
13,No,-1050,NaN,NaN,NaN,NaN,NaN,JORDAN,29.6,35,NaN,NaN


In [27]:
# Display sample of narrowed down data
worldwide_earthquake_df.tail(10)

,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,FOCAL_DEPTH,INTENSITY,COUNTRY,LATITUDE,LONGITUDE,TOTAL_DEATHS,TOTAL_INJURIES
I_D,,,,,,,,,,,,
10480,No,2020,2.0,23.0,16.0,10.0,NaN,IRAN,38.482,44.367,10.0,60.0
10483,No,2020,3.0,10.0,10.0,23.0,8.0,INDONESIA,-6.808,106.676,NaN,4.0
10485,No,2020,3.0,18.0,13.0,12.0,6.0,USA,40.751,-112.078,NaN,NaN
10486,No,2020,3.0,22.0,5.0,10.0,8.0,CROATIA,45.897,15.966,1.0,27.0
10487,Yes,2020,3.0,25.0,2.0,57.0,6.0,RUSSIA,48.986,157.693,NaN,NaN
10488,No,2020,4.0,1.0,12.0,10.0,NaN,CHINA,33.124,98.916,NaN,NaN
10490,Yes,2020,5.0,2.0,12.0,17.0,7.0,GREECE,34.205,25.712,NaN,NaN
10491,No,2020,5.0,7.0,20.0,10.0,NaN,IRAN,35.725,52.044,2.0,22.0
10494,No,2020,5.0,18.0,13.0,10.0,NaN,CHINA,27.296,103.281,4.0,24.0


In [29]:
#Total number of row and columns of the new dataframe
worldwide_earthquake_df.shape

(6193, 12)

In [28]:
# Get information about the DataFrame
worldwide_earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6193 entries, 1 to 10501
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FLAG_TSUNAMI    6193 non-null   object 
 1   YEAR            6193 non-null   int64  
 2   MONTH           5786 non-null   float64
 3   DAY             5632 non-null   float64
 4   HOUR            4151 non-null   float64
 5   FOCAL_DEPTH     3228 non-null   float64
 6   INTENSITY       2815 non-null   float64
 7   COUNTRY         6193 non-null   object 
 8   LATITUDE        6144 non-null   object 
 9   LONGITUDE       6144 non-null   object 
 10  TOTAL_DEATHS    1693 non-null   float64
 11  TOTAL_INJURIES  1253 non-null   float64
dtypes: float64(7), int64(1), object(4)
memory usage: 629.0+ KB


Handling null values: 
we narrowed down the number of columns that have the most number of null values and also we are not using them for our analysis goals.
We leave the rest of missing values as is because it is the nature or characteristics of our dataset.
Comparing the total number of data (rows) 6193 with the number of Non-Null Count of each column shows that we were able to reduce the number of null values by dropping some columns resonabally.
As it is presented in the head and the tail of the dataset the more resent the data is the less missing value is in our dataset, which is because in many years ago there is not much data recorded. 